In [20]:
import os
import csv
import requests
#from vertexai.generative_models import GenerativeModel
import vertexai
import PIL.Image
from IPython.display import Image
from IPython.core.display import HTML
import google.generativeai as genai



In [21]:
folder_path = "grad-cam/rois/"
target_objects = ["car", "person", "tree", "building", "sky", "road", "sign", "bicycle", "bus", "truck", "motorcycle", "traffic light", "unrecognizable"]
output_csv = "gemini_output.csv"

prompt = f"""
Analyze the provided image and identify the objects present. 
Specifically check if any of the following objects are present: {', '.join(target_objects)}.
If you detect other objects not in the list, please mention them as well.
The output should be ONLY a list in square brackets! Example: 
["car", "tree", "vegetation"]
"""
with open('api_key.txt', 'r') as file:
    API_KEY = file.read().strip()
genai.configure(api_key=API_KEY)


model = genai.GenerativeModel(model_name='gemini-1.5-flash')


# Collect results in memory
results = []


for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.png', '.jpeg')):  # Check for image files
        image_path = folder_path + filename
        print(f"Processing image: {image_path}")

        try:
           
            image = Image(image_path)

            # Use the model to generate text
            response = model.generate_content([prompt,image])
            #response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

            # Extract the generated text from response
            analysis_result = response.text

            if analysis_result:
                results.append([filename, analysis_result])
                print(f"Analysis result for {filename} collected.")
            else:
                results.append([filename, "Failed to analyze"])
                print(f"Failed to analyze {filename}, result collected.")

        except requests.exceptions.RequestException as e:
            print(f"Error making API request: {e}")
            results.append([filename, "Error making API request"])
        except (KeyError, IndexError) as e:
            print(f"Error extracting content from the response: {e}")
            results.append([filename, "Error extracting content"])
        except FileNotFoundError:
            print(f"Error: Image file not found at {image_path}")
            results.append([filename, "File not found"])
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            results.append([filename, "Unexpected error"])

# Write all results to CSV at once
with open(output_csv, mode='w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Image Name", "Analysis Result"])
    csv_writer.writerows(results)

print(f"Analysis complete. Results saved to {output_csv}.")


Processing image: grad-cam/rois/roi_0_0.jpeg
Analysis result for roi_0_0.jpeg collected.
Analysis complete. Results saved to gemini_output.csv.
